In [ ]:
## 加载必要的库
import numpy as np
import xarray as xr
import os, cmaps

from sklearn.feature_selection import f_regression

import matplotlib.pyplot as plt

import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.io.shapereader as shpreader
from cartopy.util import add_cyclic_point

##-----------------------------------------------------------------------------------------------------------------##
## 读数据
with xr.open_dataset('/mnt/e/research/data/seasonal/DJF/2.5x2.5/sst.DJF.mean.anom.nc') as f1:
      pre = f1['sst_anom'][:-1,:,:]
      lat, lon = f1['lat'], f1['lon']
pre2d = np.array(pre).reshape(pre.shape[0],pre.shape[1]*pre.shape[2])
del pre

with xr.open_dataset('/mnt/e/research/data/seasonal/DJF/2.5x2.5/pc.DJF.sst.nc') as f2:
      pc = f2['pc'][0,:]

del f1, f2

##-----------------------------------------------------------------------------------------------------------------##
## 回归系数
A = np.vstack([pc, np.ones(len(pc))]).T
pre_reg = np.linalg.lstsq(A, pre2d)[0][0].reshape(len(lat),len(lon))

## 相关系数
pre_cor = np.corrcoef(pre2d.T,pc)[:-1,-1].reshape(len(lat),len(lon))

## 显著性检验
pre_cor_sig = f_regression(np.nan_to_num(pre2d), pc)[1].reshape(len(lat),len(lon))
area = np.where(pre_cor_sig < 0.01)

del pre2d, pc, A, pre_cor_sig

##-----------------------------------------------------------------------------------------------------------------##
## 生成地图网格
pre_reg_cyc, lon_cyc = add_cyclic_point(pre_reg, coord = lon)
pre_cor_cyc = add_cyclic_point(pre_cor)
nx, ny = np.meshgrid(lon_cyc, lat)

del pre_reg, pre_cor, lat, lon, lon_cyc

##-----------------------------------------------------------------------------------------------------------------##
## 绘图
plt.figure(figsize = (12, 10))
plt.subplots_adjust(hspace = 0.3)

ax1 = plt.subplot(211, projection = ccrs.PlateCarree(central_longitude = 180))
ax1.coastlines(lw = 0.6)
ax1.set_global()

c1 = ax1.contourf(nx, ny, pre_reg_cyc, np.arange(-1.5,1.6,0.1), cmap = cmaps.BlWhRe, transform = ccrs.PlateCarree())
plt.colorbar(c1, shrink = 1.0, pad = 0.01)

## 显著性打点
sig1 = ax1.scatter(nx[area], ny[area], marker = '.', s = 1, c = 'k', alpha = 0.6, transform = ccrs.PlateCarree())

plt.title('Reg.', fontsize = 20)

ax1.set_xticks(np.arange(0,361,30), crs = ccrs.PlateCarree())
ax1.set_yticks(np.arange(-90,90,15), crs = ccrs.PlateCarree())
ax1.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label = False))
ax1.yaxis.set_major_formatter(LatitudeFormatter())

ax1.set_extent([0,361,-40,85], crs = ccrs.PlateCarree())

##-----------------------------------------------------------------------------------------------------------------##
ax2 = plt.subplot(212, projection = ccrs.PlateCarree(central_longitude = 180))
ax2.coastlines(lw = 0.6)
ax2.set_global()

c2 = ax2.contourf(nx, ny, pre_cor_cyc, np.arange(-1.0,1.1,0.1), cmap = cmaps.BlWhRe, transform = ccrs.PlateCarree())
plt.colorbar(c2, shrink = 1.0, pad = 0.01)

## 显著性打点
sig2 = ax2.scatter(nx[area], ny[area], marker = '.', s = 1, c = 'k', alpha = 0.6, transform = ccrs.PlateCarree())

plt.title('Cor.', fontsize = 20)

ax2.set_xticks(np.arange(0,361,30), crs = ccrs.PlateCarree())
ax2.set_yticks(np.arange(-90,90,15), crs = ccrs.PlateCarree())
ax2.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label = False))
ax2.yaxis.set_major_formatter(LatitudeFormatter())

ax2.set_extent([0,361,-40,85], crs = ccrs.PlateCarree())

##-----------------------------------------------------------------------------------------------------------------##

plt.show()

##-----------------------------------------------------------------------------------------------------------------##

In [ ]:
    import h5py
    import cmaps
    import numpy as np
    import matplotlib.pyplot as plt
    from mpl_toolkits.basemap import Basemap



    # read hdf5 file
    f = h5py.File('data/FY2E_TBB_IR1_NOM_20170912_1030.hdf','r')
    # 查看文件信息
    # print(list(f.keys()))
    data = f.get('FY2E TBB Hourly Product')
    # print(list(data.attrs.items()))
    data = np.array(data)
    '''
    处理缺值，数据里有效数据的最大值为340，最小为160
    print(list(data.attrs.items()))即可查看
    '''
    data = np.ma.masked_outside(data,160,340)
    info = f.get('NomFileInfo')[0]
    '''
    卫星的中心经度 info[4]
    中心纬度为0
    '''
    lonCenter = info[4]
    # read lonlat msg
    loc = np.fromfile('data/NOM_ITG_2288_2288(0E0N)_LE.dat',dtype=np.float32,count=-1)
    latlon = loc.reshape(2,2288,2288)
    lon = np.array(latlon[0]+lonCenter)
    lat = np.array(latlon[1])
    '''
    经纬度信息里300为缺值
    '''
    lon = np.ma.masked_where(lon>300,lon)
    lat = np.ma.masked_values(lat,300)
    # 画图
    fig = plt.figure(figsize=(18.5, 10.5))
    m=Basemap(projection='cyl',llcrnrlat=10,llcrnrlon=90,urcrnrlat=35,urcrnrlon=140)
    m.drawcoastlines(color='blue')
    m.drawstates(color='blue')
    m.drawcountries(color='blue')
    x,y = m(lon, lat) # 将lats / lons转换为地图投影坐标
    cf = m.contourf(x,y,data,levels=np.linspace(np.min(data),np.max(data),400),cmap=cmaps.MPL_RdGy )
    cbar = m.colorbar(cf ,location='right',size='3%' ,pad='2%')
    cbar.set_label('Brightness Temperature ( K )', fontsize=12, fontweight='bold')
    m.drawmeridians(np.arange(90, 140, 5),labels=[0, 0, 0, 1])
    m.drawparallels(np.arange(10, 35, 5),labels=[1, 0, 0, 0])
    plt.title('Brightness Temperature(20170912_1030) ',fontweight='bold', fontsize=14)
    plt.savefig('test.png')
    plt.show()